In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, History
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Bidirectional, TimeDistributed
from sklearn.linear_model import LinearRegression
import pandas as pd
from common import *

In [2]:
# Training data
dataset_df = pd.read_csv(f"{INPUT_DATA}MAIN_DATASET.csv")[["NO5_price", "NO5_load_actual","NO5_load_delta", "NO5_generation_actual", "NO5_generation_delta", "dato_id","date_time"]][:-7381]; print(dataset_df.shape)

display(dataset_df.tail(3))

scaler = MinMaxScaler()
print("***",dataset_df["NO5_price"].values.reshape(-1,1).shape)

dataset_df["NO5_price"] = scaler.fit_transform(dataset_df["NO5_price"].values.reshape(-1,1))
dataset_df["NO5_load_actual"] = scaler.fit_transform(dataset_df["NO5_load_actual"].values.reshape(-1,1))
dataset_df["NO5_load_delta"] = scaler.fit_transform(dataset_df["NO5_load_delta"].values.reshape(-1,1))
dataset_df["NO5_generation_actual"] = scaler.fit_transform(dataset_df["NO5_generation_actual"].values.reshape(-1,1))
dataset_df["NO5_generation_delta"] = scaler.fit_transform(dataset_df["NO5_generation_delta"].values.reshape(-1,1))

(44267, 7)


,NO5_price,NO5_load_actual,NO5_load_delta,NO5_generation_actual,NO5_generation_delta,dato_id,date_time
44264,35.69,2459,-241,8286,-4139,2021-01-20 20:00:00+01:00,2021-01-20-20
44265,32.19,2385,-308,6884,-3499,2021-01-20 21:00:00+01:00,2021-01-20-21
44266,29.49,2326,-360,5496,-2752,2021-01-20 22:00:00+01:00,2021-01-20-22


*** (44267, 1)


## SeqToVec
Returning only the last output

In [6]:
days = seperate_column_to_days(dataset_df['NO5_price'])

lookbehind = 3
horizon = 24
no_hours = lookbehind*24 + horizon# lookbehind + one horizon
hour_in_days = int(no_hours / 24)

dataset = []
for i in range(len(days)-hour_in_days+1):
    dataset.append(np.concatenate((days[i:i+hour_in_days])))

dataset = np.array(dataset)

print(dataset.shape)

1845
(1842,)


C:\Users\andre\AppData\Local\Temp/ipykernel_40532/3588044864.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dataset = np.array(dataset)


In [7]:
dataset = dataset[..., np.newaxis].astype(np.float32)

n,m,k = dataset.shape
print(n,m,k)

n_steps = m - 24
horizon = 24

train = int(0.7*n)
valid = int(0.9*n)
#train = int(0.9*n)

# (7000 time series, 50 steps each)
X_train, y_train = dataset[:train, :n_steps], dataset[:train, -horizon:, 0]
X_valid, y_valid = dataset[train:valid, :n_steps], dataset[train:valid, -horizon: 0]
X_test, y_test = dataset[valid:, :n_steps], dataset[valid:, -horizon:, 0]
#X_test, y_test = dataset[train:, :n_steps], dataset[train:, -horizon:, 0]


# Seq - to - seq does not give satisfying results

Y = np.empty((n, n_steps, horizon))
for step_ahead in range(1, horizon + 1):
    Y[:,:, step_ahead - 1] = dataset[:,step_ahead:step_ahead + n_steps, 0]

Y_train = Y[:train]
Y_valid = Y[train:valid]
Y_test = Y[valid:]


print(f"{X_train.shape=}")
print(f"{X_valid.shape=}")
print(f"{X_test.shape=}")

ValueError: setting an array element with a sequence.

In [ ]:

eta = 0.002
batch_size = 200
epochs =20
model = keras.models.Sequential([
    LSTM(25, return_sequences=True, input_shape=[None, 72]),
    LSTM(25, return_sequences=True),
    TimeDistributed(keras.layers.Dense(1))
])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.000001)

opt = tf.keras.optimizers.Adam(learning_rate=eta)
model.compile(loss="mse", optimizer=opt)

callbacks = [learning_rate_reduction]

print(X_train.shape)
print(y_train.shape)

history = model.fit(np.swapaxes(X_train, 1, 2), y_train, 
                    epochs=20, shuffle=False, 
                    batch_size=batch_size,
                    validation_data=(np.swapaxes(X_valid, 1, 2), y_valid),
                    callbacks= callbacks)   

In [ ]:
model.evaluate(np.swapaxes(X_valid, 1,2), y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
y_pred = model.predict(np.swapaxes(X_valid, 1,2))
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

## Forecasting Several Steps Ahead
Iterative method

In [ ]:
# Time series slicing parameters
nbf_features = input_window = n_steps = 72 
nbf_outputs = output_window = horizon = 24

# Creating time series traing and evaluation (test) data besed on sliding window approach
time_series, time_series_df = create_time_series_data(dataset_df,                                                                             
                                                      nbf_features, 
                                                      nbf_outputs, 
                                                      column_key="NO5_price", 
                                                      column_suffix="DA",
                                                      include_target_outputs=True)
#time_series_df.to_csv("TIMESERIES_VALIDATION.csv")


# Displaying time series data
time_series_df.index.name = "t"

print(time_series.shape)
print(time_series_df.shape)
# print(time_series_df.columns)

X_train, y_train = time_series[ :34272, :input_window], time_series[ :34272, input_window:]
X_valid, y_valid = time_series[34272:34272+6581, :input_window], time_series[34272:34272+6581, input_window:]
X_test, y_test = time_series[34272+6581:, :input_window], time_series[34272+6581:, input_window:]

X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)

print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

In [ ]:
np.random.seed(43) # not 42, as it would give the first series in the train set

X_new = X_test[-1:, :]
Y_new = np.expand_dims(y_test[-1:, :], 2)

display(dataset_df[["NO5_price", "date_time"]].tail(1))

print(f"{X_new.shape=}")
print(f"{Y_new.shape=}")

# X = X_new
X = np.swapaxes(X_new, 1,2)

print(X.shape)

for step_ahead in range(horizon):
    # y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    y_pred_one = model.predict(X[:, :, step_ahead:])

    print(f"{y_pred_one.shape=}")
    #y_pred_one = y_pred_one[:, np.newaxis, :]

    # X = np.concatenate([X, y_pred_one], axis=1)
    print(f"{y_pred_one.shape=}")

    X = np.concatenate([X, y_pred_one], axis=2)
    print(f"{X.shape=}")

print(f"{X.shape=}")
Y_pred = X[:, n_steps:]
Y_pred = X[:,:, n_steps:]

In [ ]:
Y_pred = np.swapaxes(Y_pred, 1,2)
print(Y_pred.shape)

In [ ]:
plot_multiple_forecasts(X_new, Y_new, Y_pred)
save_fig("forecast_ahead_plot")
plt.ylim(0.10,0.70)
plt.show()

mse = keras.metrics.mean_squared_error(Y_new, Y_pred)
print("MSE:",np.mean(mse))

# Linear Regression

In [ ]:
# Time series slicing parameters
nbf_features = input_window = n_steps = 72 
nbf_outputs = output_window = 24


# Creating time series traing and evaluation (test) data besed on sliding window approach
time_series, time_series_df = create_time_series_data(dataset_df,                                                                             
                                                      nbf_features, 
                                                      nbf_outputs, 
                                                      column_key="NO5_price", 
                                                      column_suffix="DA",
                                                      include_target_outputs=True)
#time_series_df.to_csv("TIMESERIES_VALIDATION.csv")


# Displaying time series data
time_series_df.index.name = "t"

print(time_series.shape)
print(time_series_df.shape)
# print(time_series_df.columns)

X_train, y_train = time_series[ :34272, :input_window], time_series[ :34272, input_window:]
X_valid, y_valid = time_series[34272:34272+6581, :input_window], time_series[34272:34272+6581, input_window:]
X_test, y_test = time_series[34272+6581:, :input_window], time_series[34272+6581:, input_window:]

# X_train = np.expand_dims(X_train, axis=2)
# X_valid = np.expand_dims(X_valid, axis=2)
# X_test = np.expand_dims(X_test, axis=2)

print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

In [ ]:
MLR_model = LinearRegression()
MLR_model.fit(X_train, y_train[:, :1])

In [ ]:

X_new = X_test[-1:, :]
Y_new = y_test[-1:, :]

X = X_new


for step_ahead in range(24):
    # y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    y_pred_one = MLR_model.predict(X[:, step_ahead:])

    print(y_pred_one)

    print(f"{y_pred_one.shape=}")
    #y_pred_one = y_pred_one[:, np.newaxis, :]

    # X = np.concatenate([X, y_pred_one], axis=1)
    print(f"{y_pred_one.shape=}")

    X = np.concatenate([X, y_pred_one], axis=1)
    print(f"{X.shape=}")

print(f"{X.shape=}")
Y_pred = X[:, n_steps:]



In [ ]:
X_new = np.expand_dims(X_new,2)
Y_new = np.expand_dims(Y_new,2)
Y_pred = np.expand_dims(Y_pred,2)

plot_multiple_forecasts(X_new, Y_new, Y_pred)
save_fig("forecast_ahead_plot")
plt.ylim(0.10,0.70)
plt.show()


mse = keras.metrics.mean_squared_error(Y_new, Y_pred)
print("MSE:",np.mean(mse))

# Own NN

In [ ]:
from models_project2 import *
eta = 0.05
lmb = 0
hidden_size = 64
act_func = "relu"
act_func = "leaky_relu"
epochs = 200

In [ ]:
own_model, tf_model = NN_simple_architecture(eta, nbf_features,
                                                         problem_type="regression",
                                                         nbf_outputs=1,
                                                         X_test=X_test.copy(), t_test=y_test.copy()[:, :1],
                                                         lmb=lmb, 
                                                         hidden_size=hidden_size, 
                                                         act_func=act_func)

own_model, tf_model = NN_large_architecture(eta, nbf_features,
                                                         problem_type="regression",
                                                         nbf_outputs=1,
                                                         X_test=X_test.copy(), t_test=y_test.copy()[:, :1],
                                                         lmb=lmb, 
                                                         hidden_size=hidden_size, 
                                                         act_func=act_func)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(y_train[:, :1])

train_losses, test_losses = own_model.fit(X_train, y_train[:, :1], 
                                                            batch_size=batch_size, epochs=epochs,
                                                            lr_scheduler=False, verbose=True)

In [ ]:
plt.figure()
x_plot = np.arange(train_losses.shape[0])
plt.plot(x_plot, train_losses)
plt.plot(x_plot, test_losses)

In [ ]:
#t_hat_test_own = own_model.predict(X)

In [ ]:

np.random.seed(43) # not 42, as it would give the first series in the train set

X_new = X_test[-1:, :]
Y_new = y_test[-1:, :]

X = X_new


for step_ahead in range(24):
    # y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    y_pred_one = own_model.predict(X[:, step_ahead:])

    print(y_pred_one)

    print(f"{y_pred_one.shape=}")
    #y_pred_one = y_pred_one[:, np.newaxis, :]

    # X = np.concatenate([X, y_pred_one], axis=1)
    print(f"{y_pred_one.shape=}")

    X = np.concatenate([X, y_pred_one], axis=1)
    print(f"{X.shape=}")

print(f"{X.shape=}")
Y_pred = X[:, n_steps:]


In [ ]:
X_new = np.expand_dims(X_new,2)
Y_new = np.expand_dims(Y_new,2)
Y_pred = np.expand_dims(Y_pred,2)
print(f"{Y_pred.shape=}")
print(f"{Y_new.shape=}")
print(f"{X_new.shape=}")


In [ ]:
plot_multiple_forecasts(X_new, Y_new, Y_pred)
save_fig("forecast_ahead_plot")
plt.ylim(0.10,0.70)
plt.show()



mse = keras.metrics.mean_squared_error(Y_new, Y_pred)
print("MSE:",np.mean(mse))